In [2]:
pip install networkx

                                              0.0/2.1 MB ? eta -:--:--
     -                                        0.1/2.1 MB 3.0 MB/s eta 0:00:01
     -----                                    0.3/2.1 MB 3.4 MB/s eta 0:00:01
     -----------                              0.6/2.1 MB 4.8 MB/s eta 0:00:01
     -------------------                      1.0/2.1 MB 5.8 MB/s eta 0:00:01
     -----------------------------            1.5/2.1 MB 6.8 MB/s eta 0:00:01
     --------------------------------------   2.0/2.1 MB 7.5 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 7.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib
from sklearn import datasets
from networkx.drawing.nx_agraph import graphviz_layout
%matplotlib inline

In [4]:
df = pd.read_excel('datos/datos_arbol.xlsx')

In [6]:
print(df)

    INGRESO  ESTRATO  CREDITO
0         1        2        0
1         5        2        0
2         1        4        1
3         6        0        0
4         8        5        1
5         4        0        0
6         3        5        1
7         6        2        0
8         3        5        1
9         3        2        0
10        9        2        1
11        1        2        0
12        3        0        0
13        1        3        0
14        7        2        0
15        9        1        1
16        8        3        1
17        7        2        0
18        6        3        0
19        9        2        1
20        0        3        0
21        2        4        1
22        3        4        1
23        1        3        0
24        0        5        1
25        5        4        1
26        9        5        1
27        0        5        1
28        0        4        1
29        3        3        0


## 1. Clasificación y la entropía como función objetivo de la clasificación (15%).
#### a. Utilizar la siguiente tabla para crear un árbol de clasificación de la variable Y a mano con 2 niveles.

In [7]:
## Calculamos la entropia de la variable estrato 

estrato_counts = df['ESTRATO'].value_counts()
total_samples = len(df)

entropy = 0
for count in estrato_counts:
    probability = count / total_samples
    entropy += -probability * math.log2(probability)

In [8]:
def calculate_conditional_entropy(df, feature, target):
    feature_values = df[feature].unique()
    entropy = 0
    
    for value in feature_values:
        subset = df[df[feature] == value]
        subset_size = len(subset)
        subset_y_counts = subset[target].value_counts()
        conditional_entropy = 0
        
        for count in subset_y_counts:
            probability = count / subset_size
            conditional_entropy += -probability * math.log2(probability)
        
        entropy += (subset_size / total_samples) * conditional_entropy
    
    return entropy

In [10]:
features = df.columns[:-1]
best_feature = None
max_information_gain = -1

for feature in features:
    information_gain = entropy - calculate_conditional_entropy(df, feature, 'CREDITO')
    
    if information_gain > max_information_gain:
        max_information_gain = information_gain
        best_feature = feature

In [11]:
subset1 = df[df[best_feature] == 0]
subset2 = df[df[best_feature] == 1]

In [12]:
subset1_entropy = 0
subset2_entropy = 0

if len(subset1) > 0:
    subset1_y_counts = subset1['CREDITO'].value_counts()
    for count in subset1_y_counts:
        probability = count / len(subset1)
        subset1_entropy += -probability * math.log2(probability)

if len(subset2) > 0:
    subset2_y_counts = subset2['CREDITO'].value_counts()
    for count in subset2_y_counts:
        probability = count / len(subset2)
        subset2_entropy += -probability * math.log2(probability)

subset1_information_gain = entropy - subset1_entropy
subset2_information_gain = entropy - subset2_entropy

In [13]:
print("Nodo raíz:", best_feature)
print("Subset 1 - Entropía:", subset1_entropy)
print("Subset 1 - Ganancia de información:", subset1_information_gain)
print("Subset 2 - Entropía:", subset2_entropy)
print("Subset 2 - Ganancia de información:", subset2_information_gain)

Nodo raíz: ESTRATO
Subset 1 - Entropía: 0.0
Subset 1 - Ganancia de información: 2.3764438290006864
Subset 2 - Entropía: 0.0
Subset 2 - Ganancia de información: 2.3764438290006864


### Entrene un árbol utilizando sklearn.tree.DecisionTreeClassifier y compare los resultados. ¿Qué puede concluir son iguales?

In [16]:
X = df.drop('CREDITO', axis=1)
y = df['CREDITO']

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
from sklearn.tree import DecisionTreeClassifier
clasificacion = DecisionTreeClassifier()

In [21]:
clasificacion.fit(X_train, y_train)

DecisionTreeClassifier()

In [26]:
y_pred = clasificacion.predict(X_test)
print(y_pred)

[1 1 0 0 1 0]


In [24]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 1.0


###  ¿Qué puede concluir son iguales?

Una precisión de 1.0 significa que el modelo clasificó correctamente todos los ejemplos en el conjunto de prueba. Esto sugiere que tanto el árbol de clasificación manual como el modelo entrenado con sklearn pudieron aprender patrones en los datos que les permitieron realizar predicciones precisas.